[View in Colaboratory](https://colab.research.google.com/github/apuayush/spam_classifier/blob/master/spam_filter.ipynb)

In [2]:
# for mounting drive with google colabs
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [1]:
!ls

datalab  drive	nltk_data


Dataset Link - https://www.kaggle.com/uciml/sms-spam-collection-dataset/data

The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam.

In [0]:
import pandas as pd
import numpy as np
import os 
import io
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [0]:
data = pd.read_csv('drive/spam.csv', encoding='latin-1')

In [0]:
from sklearn.utils import shuffle
data = shuffle(data)

In [46]:
len(data)

5572

In [47]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
2509,ham,U wake up already? Wat u doing? U picking us u...,NaN,NaN,NaN
4105,ham,K then 2marrow are you coming to class.,NaN,NaN,NaN
4931,ham,Match started.india &lt;#&gt; for 2,NaN,NaN,NaN
4742,ham,I am in hospital da. . I will return home in e...,NaN,NaN,NaN
697,ham,meet you in corporation st outside gap Û_ you...,NaN,NaN,NaN


In [48]:
data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5572 entries, 2509 to 4043
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 261.2+ KB


In [50]:
X = data['v2']
X.head()

2509    U wake up already? Wat u doing? U picking us u...
4105              K then 2marrow are you coming to class.
4931                Match started.india  &lt;#&gt;  for 2
4742    I am in hospital da. . I will return home in e...
697     meet you in corporation st outside gap Û_ you...
Name: v2, dtype: object

In [51]:
Y = data['v1']
Y.head()
Y.unique()

array(['ham', 'spam'], dtype=object)

In [52]:
Y[Y=='ham'].value_counts()

ham    4825
Name: v1, dtype: int64

In [53]:
Y[Y=='spam'].value_counts()

spam    747
Name: v1, dtype: int64

## Label Encoding 
0 - ham
1 - spam

In [0]:
Y = Y.replace('ham', 0)
Y = Y.replace('spam', 1)

In [55]:
Y.head()

2509    0
4105    0
4931    0
4742    0
697     0
Name: v1, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3)

## Stemming and Tokenization

In [57]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [0]:
def process_msg(msg):
  msg = msg.lower()
  words = word_tokenize(msg)
  
  sw = stopwords.words('english')
  words = [w for w in words if w not in sw]
  
  stem = PorterStemmer()
  
  words = [stem.stem(word) for word in words]
  return words

In [60]:
import random
k = random.randint(0, len(Y))
print("position at",k)
print("Before processing", X[k])
words = process_msg(X[k])
print("After Processing", words)

position at 1479
Before processing My trip was ok but quite tiring lor. Uni starts today but it's ok 4 me cos i'm not taking any modules but jus concentrating on my final yr project.
After Processing ['trip', 'ok', 'quit', 'tire', 'lor', '.', 'uni', 'start', 'today', "'s", 'ok', '4', 'co', "'m", 'take', 'modul', 'ju', 'concentr', 'final', 'yr', 'project', '.']


In [0]:
# Vecotizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=process_msg)

In [0]:
classifier = MultinomialNB()

In [63]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vectorizer', vectorizer), ('classifier', classifier)])
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [64]:
# Accuracy
acc = pipe.score(X_test, Y_test)
print('Model accuracy: ' + str(acc))

Model accuracy: 0.9473684210526315


In [0]:
# save model
import pickle
save_dir = 'drive/spam.pkl'

with open(save_dir, 'wb') as f:
    pickle.dump(pipe, f)

## Predictions

In [0]:
def classify(text):
  pred = pipe.predict([text])[0]
  confidence = pipe.predict_proba([text])[0][0]
  PRED = {
      0: "Not spam!",
      1: "Spam!"
  }
  print(PRED[pred],"with", confidence*100,"% confidence")

In [0]:
|